### Define Pattern

In [33]:
from pyparsing import *
import numpy as np
import dgl
import sys
sys.path.append('../../../')
from util_funcs import graph_utils

# add support for ground input '1'b0'
gnd = Literal("1'b0")
vcc = Literal("1'b1")
identifier = Word(alphas+"_", alphanums+"_"+'['+']') | gnd |vcc
input_port = Suppress("input") + delimitedList(identifier) + Suppress(";")
output_port = Suppress("output") + delimitedList(identifier) + Suppress(";")
# wire_list = Suppress("wire") + delimitedList(identifier) + Suppress(";")
# gate_type = oneOf("not and or nand nor xor")
port = Group(Suppress('.') + identifier + Suppress("(") + identifier + Suppress(")"))
gate = Group(identifier + identifier) + Suppress("(") + delimitedList(port) + Suppress(");")
# module_title = Suppress("module") + identifier + Suppress("(") + delimitedList(identifier) + Suppress(");")
# module = module_title + input_port + output_port + Optional(wire_list) + OneOrMore(gate) + Suppress("endmodule")

def build_circuit_graph_undirected(node_list,edge_list):
    g = dgl.DGLGraph()
    g.add_nodes(len(node_list))
    src, dst = tuple(zip(*edge_list))
    g.add_edges(src, dst)
    g.add_edges(dst, src)
    
    return g

### Standard Cell Library Parser

In [74]:
with open('/usr/local/share/qflow/tech/osu035/osu035_stdcells.v', 'rt') as fh:
    code_std_cells = fh.read()

module_title = Suppress("module") + identifier + Suppress("(") + delimitedList(identifier) + Suppress(");")
module_list = module_title.searchString(code_std_cells)
len(module_list)

35

### Get the netlist from file

In [78]:
import os
dir = 'verilogs'
graph_idx=0
graph_label=[]
edge_list = []
graph_indicator = []
node_attribute = []
count=0
filenames = 'csa_8bit.rtlbb.v'
for filename in os.listdir(dir):
    if (filename.endswith(".v") and ((filename.find("csa")>=0) or((filename.find("CSkipA")>=0)))):
        graph_idx+=1
        with open(dir+'/'+filename, 'rt') as fh:
            code = fh.read()
        #print(filename)
        input_port_list = input_port.searchString(code)
        output_port_list = output_port.searchString(code)
        gate_list = gate.searchString(code)

        #print(len(output_port_list))
        #print(len(input_port_list))
        #print(len(gate_list))
        gate_types={}
        for i in range(len(module_list)):
        #     print(module_list[i][0])
            gate_types[module_list[i][0]] = i+2

        gate_dict = {} #gate name mapped to unique integers
        gate_type_dict = {}
        #define gate types
        #input_port = 0
        #output_port = 1
        #other gates start from 2


        #Add gnd pin at the beginning:
        gate_dict["1'b0"] = 0
        gate_type_dict[0] = 0
        gate_dict["1'b1"] = 1
        gate_type_dict[1] = 0

        #Add input ports to gate_dict and gate_type_dict:
        for idx,in_port in enumerate(input_port_list):
            gate_dict[in_port[0]] = idx+2
            gate_type_dict[idx+2] = 0

        #Add gates to gate_dict: position[2] is output
        for idx, a_gate in enumerate(gate_list):
        #     gate_dict[a_gate[0][1]] = idx + len(input_port_list)
            gate_dict[a_gate[-1][1]] = idx + len(input_port_list) +2
            gate_type_dict[idx + len(input_port_list)+2] = gate_types[a_gate[0][0]]
        #     print(idx + len(input_port_list)+1)

        #Add output ports to gate_type_dict:
        for idx, a_gate in enumerate(output_port_list):
        #     gate_dict[a_gate[0][1]] = idx + len(input_port_list)
            gate_type_dict[idx + len(input_port_list)+len(gate_list)+2] = 1
        #     print(idx + len(input_port_list)+1)
        gate_type_dict_array=[]
        for idx in range (len(gate_type_dict)):
            gate_type_dict_array.append(gate_type_dict[idx])
        #print(gate_dict)
        

        node_list = []
        edges = []
        current_length = len(graph_indicator)
        #Add input ports and gates to node_list:
        for idx in range(len(gate_dict)):
            graph_indicator.append(graph_idx) 
            node_list.append(idx) 
        #Add connections to edge_list:
        for a_gate in gate_list:
            #print("GATE:",a_gate[1])
            for connection in a_gate[1:-1]:
                #print(connection)
                edge_list.append((current_length+gate_dict[connection[1]],current_length+gate_dict[a_gate[-1][1]]))
                edge_list.append((current_length+gate_dict[a_gate[-1][1]],current_length+gate_dict[connection[1]]))
                edges.append((gate_dict[connection[1]],gate_dict[a_gate[-1][1]]))

        existing_length = len(node_list)

        #Add output ports to node_list:
        for idx,out_port in enumerate(output_port_list):
            graph_indicator.append(graph_idx)
            node_list.append(idx+existing_length) 
            edge_list.append((current_length+gate_dict[out_port[0]],current_length+idx+existing_length))    
            edge_list.append((current_length+idx+existing_length,current_length+gate_dict[out_port[0]]))    
            edges.append((gate_dict[out_port[0]],idx+existing_length))   
            

        g = build_circuit_graph_undirected(node_list,edges)
        x = g.in_degrees().view(-1, 1).float()
        

        for idx,i in enumerate(x):
            node_attribute.append(i.item())
        filename = fh.name.split('.')
        
        filename = filename[0].split('/')
        print(filename[1])
#         np.savetxt("processed_training_data/"+filename[1]+'node_list.csv', node_list, delimiter=",", fmt='%s',header='')
        if(filename[1].find("rca")>=0):
            graph_label.append(1)
        if(filename[1].find("cla")>=0):
            graph_label.append(0)

        if(filename[1].find("csa")>=0):
            graph_label.append(0)
            
       
        if(filename[1].find("CSkipA")>=0):
            graph_label.append(1)


csa_12bit
csa_25bit
csa_41bit
CSkipA_28bit
CSkipA_22bit
CSkipA_47bit
CSkipA_16bit
CSkipA_5bit
csa_43bit
CSkipA_54bit
csa_5bit
csa_27bit
csa_10bit
csa_36bit
CSkipA_15bit
csa_16bit
CSkipA_57bit
csa_45bit
csa_32bit
csa_39bit
CSkipA_32bit
CSkipA_44bit
csa_28bit
CSkipA_26bit
csa_30bit
csa_47bit
csa_35bit
csa_18bit
CSkipA_34bit
CSkipA_37bit
csa_52bit
csa_22bit
csa_13bit
csa_4bit
CSkipA_52bit
CSkipA_50bit
csa_57bit
CSkipA_30bit
CSkipA_38bit
CSkipA_43bit
CSkipA_20bit
csa_59bit
csa_54bit
CSkipA_46bit
CSkipA_14bit
CSkipA_33bit
CSkipA_45bit
CSkipA_63bit
CSkipA_12bit
CSkipA_56bit
CSkipA_8bit
CSkipA_19bit
CSkipA_23bit
CSkipA_24bit
CSkipA_31bit
CSkipA_10bit
CSkipA_7bit
csa_42bit
csa_20bit
CSkipA_59bit
CSkipA_58bit
CSkipA_55bit
csa_17bit
CSkipA_42bit
CSkipA_9bit
csa_44bit
CSkipA_51bit
csa_8bit
csa_48bit
csa_11bit
CSkipA_25bit
csa_19bit
csa_34bit
csa_15bit
csa_61bit
csa_21bit
csa_63bit
csa_26bit
CSkipA_18bit
csa_55bit
csa_53bit
csa_29bit
csa_49bit
CSkipA_11bit
CSkipA_17bit
CSkipA_29bit
csa_7bit
CSkipA

In [79]:
for idx,(e0,e1) in enumerate(edge_list):
    e0+=1
    e1+=1
    edge_list[idx] = (e0,e1)
#     edge_list[idx][1]+=1 
    
    

np.savetxt('Adder3_A.txt', edge_list, delimiter=",", fmt='%s',header='')
np.savetxt('Adder3_graph_indicator.txt', graph_indicator, delimiter=",", fmt='%s',header='')
np.savetxt('Adder3_graph_labels.txt', graph_label, delimiter=",", fmt='%s',header='')
np.savetxt('Adder3__node_labels.txt', node_attribute, delimiter=",", fmt='%d',header='')

## End

In [69]:

print(edge_list)


[(29, 18), (18, 29), (121, 19), (19, 121), (139, 20), (20, 139), (148, 21), (21, 148), (130, 22), (22, 130), (32, 23), (23, 32), (35, 24), (24, 35), (38, 25), (25, 38), (41, 26), (26, 41), (59, 27), (27, 59), (95, 28), (28, 95), (131, 28), (28, 131), (131, 29), (29, 131), (27, 29), (29, 27), (28, 29), (29, 28), (49, 30), (30, 49), (85, 31), (31, 85), (131, 31), (31, 131), (131, 32), (32, 131), (30, 32), (32, 30), (31, 32), (32, 31), (67, 33), (33, 67), (131, 34), (34, 131), (103, 34), (34, 103), (131, 35), (35, 131), (33, 35), (35, 33), (34, 35), (35, 34), (76, 36), (36, 76), (131, 37), (37, 131), (112, 37), (37, 112), (131, 38), (38, 131), (36, 38), (38, 36), (37, 38), (38, 37), (58, 39), (39, 58), (131, 40), (40, 131), (94, 40), (40, 94), (131, 41), (41, 131), (39, 41), (41, 39), (40, 41), (41, 40), (0, 42), (42, 0), (14, 43), (43, 14), (6, 43), (43, 6), (14, 44), (44, 14), (6, 44), (44, 6), (42, 45), (45, 42), (44, 45), (45, 44), (43, 45), (45, 43), (14, 46), (46, 14), (6, 46), (46,